In [1]:
import pandas as pd
import numpy  as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import naive_bayes
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import spacy
import re

In [2]:
data_origin = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
regexp = re.compile('(?u)\\b\\w\\w+\\b')
en_nlp = spacy.load('en')
old_tokenizer = en_nlp.tokenizer
en_nlp.tokenizer = lambda string: old_tokenizer.tokens_from_list(regexp.findall(string))

In [4]:
def custom_tokenizer(document):
    doc_spacy = en_nlp(document)
    return [token.lemma_ for token in doc_spacy]

In [5]:
count_vect = CountVectorizer(tokenizer=custom_tokenizer,min_df=5,stop_words="english",ngram_range=(1,3))
X_train_counts = count_vect.fit_transform(data_origin.description)

C:\Users\masato\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: [W002] Tokenizer.from_list is now deprecated. Create a new Doc object instead and pass in the strings as the `words` keyword argument, for example:
from spacy.tokens import Doc
doc = Doc(nlp.vocab, words=[...])
  after removing the cwd from sys.path.


In [6]:
tf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [7]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [8]:
clf = MultinomialNB(alpha=0.01).fit(X_train_tfidf,data_origin.jobflag)

In [9]:
X_new_counts = count_vect.transform(data_test.description)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

C:\Users\masato\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: [W002] Tokenizer.from_list is now deprecated. Create a new Doc object instead and pass in the strings as the `words` keyword argument, for example:
from spacy.tokens import Doc
doc = Doc(nlp.vocab, words=[...])
  after removing the cwd from sys.path.


In [10]:
predicted = clf.predict(X_new_tfidf)

In [11]:
data_test['pre'] = predicted

In [12]:
data_test.to_csv('submit_alpha01stopmin5n3lemma.csv',columns=['id','pre'],header=False,index=False)

In [13]:
a = pd.DataFrame(X_train_counts.toarray(), columns=count_vect.get_feature_names())

In [14]:
a.head()

,-PRON-,-PRON- -PRON-,-PRON- business,-PRON- client,-PRON- company,-PRON- current,-PRON- customer,-PRON- datum,-PRON- need,-PRON- partner,...,work product,work project,work stakeholder,work stream,work team,workflow,workshop,world,write,write unit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
